<a href="https://colab.research.google.com/github/nhwhite212/DealingwithDataSpring2021/blob/master/3-WebAPIs_crawling/B2-IBM_Watson_Natural_Language_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interacting with the IBM Watson Natural Language Understanding API; POST vs GET

Another useful API, especially when dealing with text, is the [IBM Watson  Natural Language Understanding API](https://console.bluemix.net/catalog/services/natural-language-understanding), which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc.

We will give a couple of examples below, to understand how we can take an unstructured piece of text (either the text alone, or a URL with text), and extract some "semi-structured" representation of its content.



## /analyze call

We will first start with the `GET /analyze` API call ([documentation](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#get-analyze)), which takes as input a piece of text, and returns an analysis across various dimensions.

The call below gets as input a "text" variable, and returns back the sentiment of the text.

In [1]:
import requests
import json

def getSentiment(text):
    endpoint = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"

    # You can register and get your own credentials
    # The ones below have a quota of 1000 calls per day 
    # and can run out quickly if multiple people use these
    username = "apikey"
    #password = "xSOLcYbCwXT5bRS36huaiwO9kO07_m1EaMa769hTMpcR"
    password = "JjBk20E2nz3jxv9tUkasj1CIcyPOwhcb1uhMhkXdTWEn"
    parameters = {
        'features': 'emotion,sentiment',
        'version' : '2018-11-16',
        'text': text,
        'language' : 'en',
        # url = url_to_analyze, this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

In [2]:
# We will analyze the text below using the IBM Watson API

text = '''
I got their Egg & Cheese sandwich on a Whole Wheat Everything Bagel. 
First off, I loved loved loved the texture of the bagel itself. 
It was very chewy yet soft, which is a top feature for a NY style bagel. 
However, I thought there could've been more seasoning on top of 
the bagel as I found the bagel itself to be a bit bland. 

Speaking of bland, I thought the egg and cheese filling were also quite bland. 
This was definitely lacking salt and pepper in the eggs and the cheese didn't
really add too much flavor either, which was really disappointing! 
My mom also had the same complaint with her bagel sandwich 
(she had the egg sandwich on a blueberry bagel) so I definitely wasn't 
the only one.

'''

In [3]:
data = getSentiment(text)
data

{'emotion': {'document': {'emotion': {'anger': 0.104206,
    'disgust': 0.152643,
    'fear': 0.068719,
    'joy': 0.621954,
    'sadness': 0.484095}}},
 'language': 'en',
 'sentiment': {'document': {'label': 'negative', 'score': -0.600662}},
 'usage': {'features': 2, 'text_characters': 707, 'text_units': 1}}

Now, let's try to understand the structure of the answer. First, we check the high-level keys.

In [4]:
data.keys()

dict_keys(['usage', 'sentiment', 'language', 'emotion'])

Now, let's check the content of these keys:

In [5]:
data['language']

'en'

In [6]:
data['sentiment']

{'document': {'label': 'negative', 'score': -0.600662}}

In [7]:
data['emotion']

{'document': {'emotion': {'anger': 0.104206,
   'disgust': 0.152643,
   'fear': 0.068719,
   'joy': 0.621954,
   'sadness': 0.484095}}}

In [8]:
# Let's go deeper into the 'emotion' dictionary
data['emotion']['document']

{'emotion': {'anger': 0.104206,
  'disgust': 0.152643,
  'fear': 0.068719,
  'joy': 0.621954,
  'sadness': 0.484095}}

In [9]:
# And a bit more
data['emotion']['document']['emotion']

{'anger': 0.104206,
 'disgust': 0.152643,
 'fear': 0.068719,
 'joy': 0.621954,
 'sadness': 0.484095}

### Exercise

Type your own piece of text, and analyze it to extract sentiment and emotions. Discuss your findings

## Entities call

[Full Documentation of the call](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#entities)

This is a an API call that extracts entities from the text, and also the sentiment and emotion for each of these entities.

There are two new technical aspects with this API. First of all, we use the command `requests.post` as opposed to `requests.get`. This happens because `GET` is designed to handle limited amount of data. When we have a large volume of data to send as parameters, then the HTTP protocol requires the use of the `POST` command. You will also see that the parameters that we pass are not "flat" as they used to be. Instead we submit the `watson_options` as the set of parameters, which is itself semi-structured.

In terms of natural language processing, we will examine a couple of capabilities of the API. First, you will see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Trump" and "Donald Trump" get mapped to the same Knowledge Graph entity.

In [10]:
import requests
import json

def processURL(url_to_analyze):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2018-11-16',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    watson_options = {
      "url": url_to_analyze,
      "features": {
        "entities": {
          "sentiment": True,
          "emotion": True,
          "limit": 10
        }
      }
    }
    username = "apikey"
    password = "xSOLcYbCwXT5bRS36huaiwO9kO07_m1EaMa769hTMpcR"

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    return resp.json()

In [11]:
url_to_analyze = 'https://www.reuters.com/article/us-turkey-greece/erdogan-says-greece-sowing-chaos-in-mediterranean-idUSKBN25K25M/'

data = processURL(url_to_analyze)

In [12]:
# Let's see what we get back as top-level attributes
data.keys()

dict_keys(['usage', 'retrieved_url', 'language', 'entities'])

In [13]:
# Let' see the entities list
data["entities"]

[{'count': 7,
  'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Greece',
   'name': 'Greece',
   'subtype': ['GovernmentalJurisdiction', 'WineRegion', 'Country']},
  'emotion': {'anger': 0.20288,
   'disgust': 0.157348,
   'fear': 0.123059,
   'joy': 0.148242,
   'sadness': 0.163256},
  'relevance': 0.849427,
  'sentiment': {'label': 'negative', 'score': -0.188959},
  'text': 'Greece',
  'type': 'Location'},
 {'count': 4,
  'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Turkey',
   'name': 'Turkey',
   'subtype': ['Brand',
    'GovernmentalJurisdiction',
    'ProjectParticipant',
    'Country']},
  'emotion': {'anger': 0.482006,
   'disgust': 0.049143,
   'fear': 0.061887,
   'joy': 0.17263,
   'sadness': 0.230155},
  'relevance': 0.425434,
  'sentiment': {'label': 'negative', 'score': -0.369139},
  'text': 'Turkey',
  'type': 'Location'},
 {'count': 4,
  'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Recep_Tayyip_Erdoğan',


In [14]:
# Let' see the third entity. Notice the "disambiguated" attribute that
# points to "canonical" versions of the entity, in DBPedia, Freebase, OpenCYC, YAGO, etc
data["entities"][2]

{'count': 4,
 'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Recep_Tayyip_Erdoğan',
  'name': 'Recep Tayyip Erdoğan',
  'subtype': ['Politician']},
 'emotion': {'anger': 0.162606,
  'disgust': 0.028801,
  'fear': 0.037381,
  'joy': 0.487219,
  'sadness': 0.194969},
 'relevance': 0.413638,
 'sentiment': {'label': 'negative', 'score': -0.440756},
 'text': 'Tayyip Erdogan',
 'type': 'Person'}

In [15]:
# This function takes as input the result
# from the IBM Watson API and returns a list
# of entities that are relevant (above threshold)
# to the article
def getEntities(data, threshold):
    result = []
    for entity in data["entities"]:
        relevance = float(entity['relevance'])
        if relevance > threshold:
            result.append(entity['text'])
    return result

getEntities(data, 0.25)

['Greece',
 'Turkey',
 'Tayyip Erdogan',
 'Turkish navy',
 'Mediterranean Sea',
 'Mediterranean',
 'Navtex',
 'Athens',
 'Navtex']

### Exercise

* First of all, **get your own credentials for the IBM Watson API**. The demo key that we use above has a limited quota.
* Use an API to get news articles. 
    * Option 1: Use the API at https://newsapi.org to fetch the news from various sources. Print the entities that are currently being discussed in the news, together with their relevance value and the associated sentiment.
    * Option 2: Use the NY Times API to fetch the Top Stories News. You can register and get an API key at https://developer.nytimes.com/. The `Top Stories V2 API` provides the details of the news of the day: (The API call documentation is at https://developer.nytimes.com/docs/top-stories-product/1/overview and the API Call is  https://api.nytimes.com/svc/topstories/v2/home.json?api-key=PUTYOURKEYHERE). Repeat the entity extraction process from above.
    * Option 3: Use the Guardian API at https://open-platform.theguardian.com/documentation/ to fetch news from The Guardian.
